<a href="https://colab.research.google.com/github/gonzalocandia92/Laboratorio-1---Analisis-de-Datos/blob/main/Laboratorio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Laboratorio de Analisis de Datos

# ETL

##Extracción

In [1]:
from google.colab import drive
import pandas as pd
import chardet
import os
import numpy as np
from datetime import datetime

# ---------- MONTAJE DE GOOGLE DRIVE ----------
drive.mount('/content/drive', force_remount=True)

# ---------- RUTAS DE LOS ARCHIVOS ----------
ruta_base = '/content/drive/MyDrive/analisis_de_datos-info/Laboratorio/'  # ← ajusta si los guardaste en otra carpeta

archivos = {
    'clientes': f'{ruta_base}clientes.csv',
    'ventas': f'{ruta_base}ventas_mayorista.csv'
}

# ---------- FUNCIÓN DE CARGA ----------
def cargar_csv(ruta_archivo):
    if not os.path.exists(ruta_archivo):
        print(f'❌ El archivo no se encuentra en la ruta: {ruta_archivo}')
        return None
    try:
        with open(ruta_archivo, 'rb') as f:
            result = chardet.detect(f.read(100000))
        df = pd.read_csv(ruta_archivo, encoding=result['encoding'])
        print(f'✅ Extracción exitosa de: {os.path.basename(ruta_archivo)}')
        print(f'   Encoding detectado: {result["encoding"]}')
        return df
    except Exception as e:
        print(f'❌ Error al leer {ruta_archivo}: {e}')
        return None

# ---------- EXTRACCIÓN ----------
df_clientes = cargar_csv(archivos['clientes'])
df_ventas = cargar_csv(archivos['ventas'])

# ---------- VERIFICACIÓN ----------
if df_clientes is not None and df_ventas is not None:
    print('\n✅ Ambos archivos fueron cargados correctamente.\n')
    print('Ejemplo clientes:')
    display(df_clientes.head(10))
    print('\nEjemplo ventas:')
    display(df_ventas.head(10))
else:
    raise Exception("❌ No se pudieron cargar todos los archivos.")

Mounted at /content/drive
✅ Extracción exitosa de: clientes.csv
   Encoding detectado: utf-8
✅ Extracción exitosa de: ventas_mayorista.csv
   Encoding detectado: utf-8

✅ Ambos archivos fueron cargados correctamente.

Ejemplo clientes:


,id_cliente,nombre_cliente,provincia,localidad,categoria_cliente
0,1,Julieta Torres,Chaco,Quitilipi,distribuidor
1,2,Romina Martínez,Chaco,Charata,supermercado
2,3,Nadia Villalba,Misiones,Leandro N. Alem,restaurante
3,4,Jorge Acosta,Misiones,San Vicente,kiosco
4,5,Florencia García,Chaco,Charata,kiosco
5,6,Diego Valdez,Misiones,Eldorado,restaurante
6,7,Jorge Villalba,Chaco,Villa Ángela,almacén
7,8,Ivana Fernández,Formosa,Laguna Blanca,almacén
8,9,Ezequiel Romero,Formosa,Riacho He-Hé,almacén
9,10,Elena Gómez,Misiones,San Vicente,almacén



Ejemplo ventas:


,fecha_hora,id_cliente,nombre_cliente,producto,categoria_producto,cantidad,precio_unitario,total
0,2025-11-08 20:29:51,521,Damián Castro,Azúcar 1kg,mercaderia,5,500,2500
1,2024-12-16 11:56:27,806,Hernán Fernández,Pack fideos 500g,mercaderia,5,450,2250
2,2022-04-11 10:05:47,2259,Nadia Sosa,Agua mineral 2L,bebidas,4,600,2400
3,2024-06-21 09:13:20,703,Juan Fernández,Pack fideos 500g,mercaderia,4,450,1800
4,2025-02-05 18:48:48,2007,Rocío Benítez,Vino tinto 750ml,bebidas,5,1500,7500
5,2023-03-20 17:53:30,4716,Ariel Pérez,Cerveza lata 473ml,bebidas,3,350,1050
6,2023-03-16 10:29:55,2036,Damián Domínguez,Agua mineral 2L,bebidas,2,600,1200
7,2025-01-06 11:43:18,819,Laura Flores,Agua mineral 2L,bebidas,3,600,1800
8,2024-05-11 22:35:12,4954,Nicolás Domínguez,Azúcar 1kg,mercaderia,1,500,500
9,2024-02-28 17:40:27,2605,Agustín Medina,Azúcar 1kg,mercaderia,4,500,2000


In [2]:
print(df_clientes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id_cliente         5000 non-null   int64 
 1   nombre_cliente     5000 non-null   object
 2   provincia          5000 non-null   object
 3   localidad          5000 non-null   object
 4   categoria_cliente  5000 non-null   object
dtypes: int64(1), object(4)
memory usage: 195.4+ KB
None


In [3]:
print(df_ventas.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   fecha_hora          100000 non-null  object
 1   id_cliente          100000 non-null  int64 
 2   nombre_cliente      100000 non-null  object
 3   producto            100000 non-null  object
 4   categoria_producto  100000 non-null  object
 5   cantidad            100000 non-null  int64 
 6   precio_unitario     100000 non-null  int64 
 7   total               100000 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 6.1+ MB
None


## Transfromación

In [4]:
# ---------- LIMPIEZA DE COLUMNAS ----------
def limpiar_columnas(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')
    return df

df_clientes = limpiar_columnas(df_clientes)
df_ventas = limpiar_columnas(df_ventas)

# ---------- LIMPIEZA DE TEXTO ----------
columnas_texto = ['nombre_cliente', 'provincia', 'localidad', 'categoria_cliente', 'producto', 'categoria_producto']
for col in columnas_texto:
    if col in df_clientes.columns:
        df_clientes[col] = df_clientes[col].astype(str).str.strip().str.upper()
    if col in df_ventas.columns:
        df_ventas[col] = df_ventas[col].astype(str).str.strip().str.upper()

# ---------- CONVERSIÓN DE FECHA ----------
df_ventas['fecha_hora'] = pd.to_datetime(df_ventas['fecha_hora'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

if df_ventas['fecha_hora'].isna().sum() > 0:
    print("⚠️ Algunas fechas no se pudieron convertir")
else:
    print("✅ Todas las fechas convertidas correctamente")

# ---------- DUPLICADOS Y NULOS ----------
print(f"\nDuplicados en clientes (id_cliente): {df_clientes['id_cliente'].duplicated().sum()}")
print(f"Duplicados en ventas (filas completas): {df_ventas.duplicated().sum()}")

print("\nNulos en clientes:")
print(df_clientes.isnull().sum())
print("\nNulos en ventas:")
print(df_ventas.isnull().sum())



✅ Todas las fechas convertidas correctamente

Duplicados en clientes (id_cliente): 0
Duplicados en ventas (filas completas): 0

Nulos en clientes:
id_cliente           0
nombre_cliente       0
provincia            0
localidad            0
categoria_cliente    0
dtype: int64

Nulos en ventas:
fecha_hora            0
id_cliente            0
nombre_cliente        0
producto              0
categoria_producto    0
cantidad              0
precio_unitario       0
total                 0
dtype: int64


In [5]:
# ---------- CONVERSIÓN DE CAMPOS MONETARIOS A FLOAT ----------
print("\nConversión de campos monetarios a tipo float...")

# Lista de columnas que deben ser numéricas con decimales
columnas_float = ['precio_unitario', 'total']

for col in columnas_float:
    if col in df_ventas.columns:
        # Forzamos a numérico, convirtiendo errores en NaN
        df_ventas[col] = pd.to_numeric(df_ventas[col], errors='coerce')

        # Si quedaron nulos (por error), los rellenamos con 0 (poco probable en tus datos)
        if df_ventas[col].isna().sum() > 0:
            print(f"   Advertencia: {df_ventas[col].isna().sum()} valores no numéricos en {col}. Se convierten a 0.")
            df_ventas[col] = df_ventas[col].fillna(0)

        # Finalmente convertimos a float
        df_ventas[col] = df_ventas[col].astype(float)
        print(f"   {col} → convertido a float64 correctamente")
    else:
        print(f"   Advertencia: columna {col} no encontrada en df_ventas")

# También convertimos 'cantidad' a entero (por si acaso)
if 'cantidad' in df_ventas.columns:
    df_ventas['cantidad'] = pd.to_numeric(df_ventas['cantidad'], errors='coerce').fillna(0).astype(int)
    print(f"   cantidad → convertido a int64")





Conversión de campos monetarios a tipo float...
   precio_unitario → convertido a float64 correctamente
   total → convertido a float64 correctamente
   cantidad → convertido a int64


In [6]:
print(df_ventas.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   fecha_hora          100000 non-null  datetime64[ns]
 1   id_cliente          100000 non-null  int64         
 2   nombre_cliente      100000 non-null  object        
 3   producto            100000 non-null  object        
 4   categoria_producto  100000 non-null  object        
 5   cantidad            100000 non-null  int64         
 6   precio_unitario     100000 non-null  float64       
 7   total               100000 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 6.1+ MB
None


In [7]:
# ---------- TRANSFORMACIÓN ROBUSTA DE FECHAS (MAYORISTA) ----------
print("\nIniciando transformación ROBUSTA de la columna 'fecha_hora'...")

# Copia de respaldo por seguridad
df_ventas_original = df_ventas.copy()

# INTENTO 1: Formato principal esperado del script generador
df_ventas['fecha_hora'] = pd.to_datetime(
    df_ventas['fecha_hora'],
    format='%Y-%m-%d %H:%M:%S',  # ← Este es el formato exacto de tu script
    errors='coerce'
)

restantes = df_ventas['fecha_hora'].isna().sum()
print(f"   → Primer intento: {len(df_ventas) - restantes:,} fechas convertidas correctamente")

# FUNCIÓN MÁGICA: intenta MÁS DE 15 formatos diferentes
def convertir_fecha_segura(valor):
    if pd.isna(valor) or str(valor).strip() == '':
        return pd.NaT

    formatos = [
        '%Y-%m-%d %H:%M:%S',     # 2023-05-12 14:30:22 ← tu formato real
        '%Y/%m/%d %H:%M:%S',     # 2023/05/12 14:30:22
        '%d/%m/%Y %H:%M:%S',     # 12/05/2023 14:30:22
        '%d-%m-%Y %H:%M:%S',     # 12-05-2023 14:30:22
        '%Y-%m-%d %H:%M',        # 2023-05-12 14:30
        '%d/%m/%Y %H:%M',        # 12/05/2023 14:30
        '%d-%m-%Y %H:%M',        # 12-05-2023 14:30
        '%Y/%m/%d %H:%M',        # 2023/05/12 14:30
        '%Y-%m-%d',              # solo fecha: 2023-05-12
        '%d/%m/%Y',              # 12/05/2023
        '%d-%m-%Y',              # 12-05-2023
        '%Y/%m/%d',              # 2023/05/12
        '%d/%m/%y %H:%M:%S',     # 12/05/23 14:30:22
        '%d/%m/%y %H:%M',        # 12/05/23 14:30
    ]

    valor_str = str(valor).strip()
    for fmt in formatos:
        try:
            return datetime.strptime(valor_str, fmt)
        except ValueError:
            continue
    return pd.NaT

# Si quedó alguna fecha sin convertir → la recuperamos con la función mágica
if restantes > 0:
    print(f"   Intentando recuperar {restantes:,} fechas problemáticas con múltiples formatos...")
    mask_nat = df_ventas['fecha_hora'].isna()
    recuperadas = df_ventas_original.loc[mask_nat, 'fecha_hora'].apply(convertir_fecha_segura)
    df_ventas.loc[mask_nat, 'fecha_hora'] = recuperadas

    nuevas_restantes = recuperadas.isna().sum()
    print(f"   Recuperadas: {restantes - nuevas_restantes:,}")
    if nuevas_restantes > 0:
        print(f"   Aún sin convertir: {nuevas_restantes:,}")

# Conteo final
final_ok = df_ventas['fecha_hora'].notna().sum()
final_error = df_ventas['fecha_hora'].isna().sum()

print(f"\nFECHAS CONVERTIDAS CORRECTAMENTE: {final_ok:,}")
print(f"FECHAS CON ERROR (NaT): {final_error:,}")

# GUARDAR ERRORES (si los hay)
if final_error > 0:
    errores = df_ventas_original[df_ventas['fecha_hora'].isna()].copy()
    ruta_errores = '/content/drive/MyDrive/analisis_de_datos-info/Laboratorio/errores_fechas_mayorista.csv'
    errores.to_csv(ruta_errores, sep=';', index=False, encoding='utf-8-sig')
    print(f"   Errores guardados en: {ruta_errores}")

# FORZAR TIPO FINAL datetime64[ns]
df_ventas['fecha_hora'] = pd.to_datetime(df_ventas['fecha_hora'], errors='coerce')

print(f"\nTIPO FINAL de 'fecha_hora': {df_ventas['fecha_hora'].dtype}")
print("Ejemplo de fechas transformadas:")
display(df_ventas['fecha_hora'].head(10))

print("\nTRANSFORMACIÓN DE FECHAS COMPLETADA CON ÉXITO")


Iniciando transformación ROBUSTA de la columna 'fecha_hora'...
   → Primer intento: 100,000 fechas convertidas correctamente

FECHAS CONVERTIDAS CORRECTAMENTE: 100,000
FECHAS CON ERROR (NaT): 0

TIPO FINAL de 'fecha_hora': datetime64[ns]
Ejemplo de fechas transformadas:


,fecha_hora
0,2025-11-08 20:29:51
1,2024-12-16 11:56:27
2,2022-04-11 10:05:47
3,2024-06-21 09:13:20
4,2025-02-05 18:48:48
5,2023-03-20 17:53:30
6,2023-03-16 10:29:55
7,2025-01-06 11:43:18
8,2024-05-11 22:35:12
9,2024-02-28 17:40:27



TRANSFORMACIÓN DE FECHAS COMPLETADA CON ÉXITO


In [8]:
# ---------- MERGE CON TABLA MAESTRA DE CLIENTES ----------
df_final = pd.merge(
    df_ventas,
    df_clientes[['id_cliente', 'nombre_cliente', 'provincia', 'localidad', 'categoria_cliente']],
    on='id_cliente',
    how='left',
    suffixes=('', '_master')
)

# Si hay inconsistencias entre ventas y clientes, las detectamos
# Solo verificamos 'nombre_cliente' ya que 'provincia', 'localidad' y 'categoria_cliente' solo existen en df_clientes
inconsistencias = df_final[
    (df_final['nombre_cliente'] != df_final['nombre_cliente_master'])
]

if len(inconsistencias) == 0:
    print("✅ Perfecta consistencia de nombres de clientes entre ventas y maestra.")
    # No es necesario actualizar, ya son consistentes
else:
    print(f"⚠️ Se encontraron {len(inconsistencias)} inconsistencias en nombres de clientes → se prioriza maestra")
    # Usamos los datos de la maestra para nombre_cliente
    df_final['nombre_cliente'] = df_final['nombre_cliente_master']

# Las columnas 'provincia', 'localidad', 'categoria_cliente' ya provienen de la maestra directamente
# y no tienen contraparte en df_ventas para comparar.

df_final = df_final.drop(columns=[col for col in df_final.columns if col.endswith('_master')])

print(f"\nDataset final: {len(df_final):,} registros")
display(df_final.head())



✅ Perfecta consistencia de nombres de clientes entre ventas y maestra.

Dataset final: 100,000 registros


,fecha_hora,id_cliente,nombre_cliente,producto,categoria_producto,cantidad,precio_unitario,total,provincia,localidad,categoria_cliente
0,2025-11-08 20:29:51,521,DAMIÁN CASTRO,AZÚCAR 1KG,MERCADERIA,5,500.0,2500.0,CHACO,FONTANA,RESTAURANTE
1,2024-12-16 11:56:27,806,HERNÁN FERNÁNDEZ,PACK FIDEOS 500G,MERCADERIA,5,450.0,2250.0,CORRIENTES,ESQUINA,DISTRIBUIDOR
2,2022-04-11 10:05:47,2259,NADIA SOSA,AGUA MINERAL 2L,BEBIDAS,4,600.0,2400.0,CHACO,GENERAL SAN MARTÍN,ALMACÉN
3,2024-06-21 09:13:20,703,JUAN FERNÁNDEZ,PACK FIDEOS 500G,MERCADERIA,4,450.0,1800.0,CHACO,PUERTO VILELAS,KIOSCO
4,2025-02-05 18:48:48,2007,ROCÍO BENÍTEZ,VINO TINTO 750ML,BEBIDAS,5,1500.0,7500.0,CORRIENTES,CORRIENTES,RESTAURANTE


##CARGA

In [9]:
# ---------- CARGA FINAL ----------
ruta_salida = '/content/drive/MyDrive/analisis_de_datos-info/Laboratorio/datos_mayorista_transformados.csv'
df_final.to_csv(ruta_salida, sep=';', index=False, encoding='utf-8-sig')
print(f"\nDataset guardado en: {ruta_salida}")
print(f"Rango de fechas: {df_final['fecha_hora'].min()} → {df_final['fecha_hora'].max()}")


Dataset guardado en: /content/drive/MyDrive/analisis_de_datos-info/Laboratorio/datos_mayorista_transformados.csv
Rango de fechas: 2022-01-01 07:33:10 → 2025-12-28 21:48:46


#EDA




In [10]:
# ===================================
# EDA - ANÁLISIS EXPLORATORIO
# ===================================

print(df_final.info())

print("\nValores faltantes:")
print(df_final.isnull().sum().sort_values(ascending=False))

print(f"\nRegistros duplicados: {df_final.duplicated().sum()}")

print("\nEstadísticas descriptivas numéricas:")
display(df_final.describe())

print("\nTop 10 localidades con más facturación:")
display(df_final.groupby('localidad')['total'].sum().sort_values(ascending=False).head(10))

print("\nTop 10 clientes con mayor facturación:")
display(df_final.groupby(['id_cliente', 'nombre_cliente'])['total'].sum().sort_values(ascending=False).head(10))

print("\nFacturación por categoría de cliente:")
display(df_final.groupby('categoria_cliente')['total'].agg(['count', 'sum', 'mean']).round(2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   fecha_hora          100000 non-null  datetime64[ns]
 1   id_cliente          100000 non-null  int64         
 2   nombre_cliente      100000 non-null  object        
 3   producto            100000 non-null  object        
 4   categoria_producto  100000 non-null  object        
 5   cantidad            100000 non-null  int64         
 6   precio_unitario     100000 non-null  float64       
 7   total               100000 non-null  float64       
 8   provincia           100000 non-null  object        
 9   localidad           100000 non-null  object        
 10  categoria_cliente   100000 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(6)
memory usage: 8.4+ MB
None

Valores faltantes:
fecha_hora            0
id_cliente         

,fecha_hora,id_cliente,cantidad,precio_unitario,total
count,100000,100000.000000,100000.000000,100000.000000,100000.000000
mean,2024-01-01 16:54:15.656320,2501.197710,3.003370,715.088500,2146.758000
min,2022-01-01 07:33:10,1.000000,1.000000,350.000000,350.000000
25%,2023-01-02 13:43:36,1249.000000,2.000000,450.000000,1000.000000
50%,2024-01-05 08:34:52,2492.000000,3.000000,600.000000,1800.000000
75%,2025-01-02 19:03:06,3762.000000,4.000000,900.000000,2800.000000
max,2025-12-28 21:48:46,5000.000000,5.000000,1500.000000,7500.000000
std,NaN,1445.205812,1.413456,361.512751,1567.044271



Top 10 localidades con más facturación:


,total
localidad,
QUITILIPI,6556000.0
PIRANÉ,6475650.0
COMANDANTE FONTANA,6365000.0
BELLA VISTA,6151600.0
PUERTO IGUAZÚ,5952950.0
MONTECARLO,5930800.0
HERRADURA,5923450.0
BARRANQUERAS,5847350.0
JARDÍN AMÉRICA,5800300.0



Top 10 clientes con mayor facturación:


,,total
id_cliente,nombre_cliente,
1551,ANA ÁLVAREZ,92450.0
1071,JUAN SILVA,90050.0
2799,JORGE FERNÁNDEZ,88950.0
3413,NICOLÁS GÓMEZ,85400.0
543,JUAN LÓPEZ,85250.0
267,LUCÍA DUARTE,85000.0
3902,JORGE MEDINA,83100.0
1003,ARIEL MEDINA,82950.0
1250,DANIEL GARCÍA,81700.0



Facturación por categoría de cliente:


,count,sum,mean
categoria_cliente,,,
ALMACÉN,19783,42664550.0,2156.63
DISTRIBUIDOR,19613,42148100.0,2148.99
KIOSCO,20899,44799650.0,2143.63
RESTAURANTE,19423,41811000.0,2152.65
SUPERMERCADO,20282,43252500.0,2132.56



###Rentabilidad y ventas

*  KPI principales: Monto total, Monto promedio por transacción, Transacciones por
cliente.
*   Identificar productos con mayor margen




In [11]:
# ===================================================
# 4) ANÁLISIS DE NEGOCIO - PUNTOS A y B (CONCRETOS)
# ===================================================

# Aseguramos columnas temporales útiles
df_final['fecha'] = df_final['fecha_hora'].dt.date
df_final['año_mes'] = df_final['fecha_hora'].dt.to_period('M')

print("="*60)
print("ANÁLISIS DE NEGOCIO")
print("="*60)

# ==================================================================
# A) RENTABILIDAD Y VENTAS
# ==================================================================
print("\nA) RENTABILIDAD Y VENTAS\n" + "-"*50)

# KPI PRINCIPALES
total_ventas = df_final['total'].sum()
n_transacciones = len(df_final)
n_clientes = df_final['id_cliente'].nunique()

kpi = pd.Series({
    'Monto Total Facturado': f"${total_ventas:,.0f}",
    'Ticket Promedio por Transacción': f"${df_final['total'].mean():,.0f}",
    'Transacciones Totales': f"{n_transacciones:,}",
    'Clientes Únicos': f"{n_clientes:,}",
    'Transacciones por Cliente (promedio)': f"{n_transacciones/n_clientes:.1f}"
})
print("KPI PRINCIPALES")
display(kpi.to_frame().T)

# Productos con mayor contribución
productos_rentabilidad = df_final.groupby('producto').agg(
    facturacion_total=('total', 'sum'),
    unidades_vendidas=('cantidad', 'sum'),
    precio_promedio_unitario=('precio_unitario', 'mean'),
    transacciones=('total', 'count')
).round(2)

productos_rentabilidad['contribución_%'] = (productos_rentabilidad['facturacion_total'] / total_ventas * 100).round(2)
productos_rentabilidad = productos_rentabilidad.sort_values('facturacion_total', ascending=False)

print("\nTop 5 productos con mayor contribución a la facturación")
top5_productos = productos_rentabilidad.head(5).copy()
top5_productos['facturacion_total'] = top5_productos['facturacion_total'].map('{:,.0f}'.format)
top5_productos['precio_promedio_unitario'] = top5_productos['precio_promedio_unitario'].map('${:,.0f}'.format)
display(top5_productos[['facturacion_total', 'unidades_vendidas', 'precio_promedio_unitario', 'contribución_%', 'transacciones']])

print("\nRECOMENDACIÓN A (Rentabilidad y ventas):")
print("→ Priorizar stock permanente y negociar mejores condiciones de compra con proveedores de:")
for i, prod in enumerate(productos_rentabilidad.head(5).index, 1):
    contrib = productos_rentabilidad.loc[prod, 'contribución_%']
    print(f"   {i}. {prod} ({contrib}% de la facturación total)")


ANÁLISIS DE NEGOCIO

A) RENTABILIDAD Y VENTAS
--------------------------------------------------
KPI PRINCIPALES


,Monto Total Facturado,Ticket Promedio por Transacción,Transacciones Totales,Clientes Únicos,Transacciones por Cliente (promedio)
0,"$214,675,800","$2,147","100,000","5,000",20.0



Top 5 productos con mayor contribución a la facturación


,facturacion_total,unidades_vendidas,precio_promedio_unitario,contribución_%,transacciones
producto,,,,,
VINO TINTO 750ML,"64,422,000",42948,"$1,500",30.01,14324
GASEOSA COLA 2.25L,"38,870,100",43189,$900,18.11,14381
ARROZ 1KG,"29,986,600",42838,$700,13.97,14269
AGUA MINERAL 2L,"25,723,800",42873,$600,11.98,14286
AZÚCAR 1KG,"21,465,500",42931,$500,10.00,14307



RECOMENDACIÓN A (Rentabilidad y ventas):
→ Priorizar stock permanente y negociar mejores condiciones de compra con proveedores de:
   1. VINO TINTO 750ML (30.01% de la facturación total)
   2. GASEOSA COLA 2.25L (18.11% de la facturación total)
   3. ARROZ 1KG (13.97% de la facturación total)
   4. AGUA MINERAL 2L (11.98% de la facturación total)
   5. AZÚCAR 1KG (10.0% de la facturación total)


###Logística y distribución


*   Identificar localidades con alta demanda para priorizar rutas y stock.
*   Recomendación: crear rutas optimizadas para las 3 localidades con mayor volumen para reducir tiempos y costos.






In [12]:
# ==================================================================
# B) LOGÍSTICA Y DISTRIBUCIÓN
# ==================================================================
print(" LOGÍSTICA Y DISTRIBUCIÓN\n" + "-"*50)


# Localidades con mayor demanda (volumen en pesos y cantidad de pedidos)
localidades_demanda = df_final.groupby('localidad').agg(
    facturacion=('total', 'sum'),
    n_pedidos=('total', 'count'),
    n_clientes_unicos=('id_cliente', 'nunique')
).round(0).sort_values('facturacion', ascending=False)

print("\nTop 10 localidades con mayor facturación (prioridad logística)")
top10_localidades = localidades_demanda.head(10).copy()
top10_localidades['facturacion'] = top10_localidades['facturacion'].map('{:,.0f}'.format)
top10_localidades = top10_localidades.rename(columns={
    'facturacion': 'Facturación Total',
    'n_pedidos': 'N° Pedidos',
    'n_clientes_unicos': 'Clientes Únicos'
})
display(top10_localidades)

# Las 3 localidades críticas para rutas optimizadas
top3_localidades = localidades_demanda.head(3)
print(f"\nLas 3 localidades con mayor volumen son:")
for i, (loc, row) in enumerate(top3_localidades.iterrows(), 1):
    print(f"   {i}. {loc.upper()} → ${row['facturacion']:,.0f} ({row['n_pedidos']:,} pedidos)")

print("\nRECOMENDACIÓN  (Logística y distribución):")
print("→ Crear rutas diarias/semanales optimizadas para las siguientes 3 localidades:")
for i, loc in enumerate(top3_localidades.index, 1):
    print(f"   {i}. {loc.upper()}")
print("→ Asignar stock dedicado y vehículo exclusivo en estas zonas para reducir tiempos de entrega de 24-48h a mismo día.")
print("→ Implementar alertas automáticas de stock bajo en estas localidades clave.")
print("→ Ofrecer entrega gratis o bonificación a clientes recurrentes en estas zonas para aumentar frecuencia.")

 LOGÍSTICA Y DISTRIBUCIÓN
--------------------------------------------------

Top 10 localidades con mayor facturación (prioridad logística)


,Facturación Total,N° Pedidos,Clientes Únicos
localidad,,,
QUITILIPI,"6,556,000",2973,152
PIRANÉ,"6,475,650",2968,149
COMANDANTE FONTANA,"6,365,000",2909,142
BELLA VISTA,"6,151,600",2915,143
PUERTO IGUAZÚ,"5,952,950",2721,135
MONTECARLO,"5,930,800",2771,138
HERRADURA,"5,923,450",2719,136
BARRANQUERAS,"5,847,350",2790,136
JARDÍN AMÉRICA,"5,800,300",2721,135



Las 3 localidades con mayor volumen son:
   1. QUITILIPI → $6,556,000 (2,973.0 pedidos)
   2. PIRANÉ → $6,475,650 (2,968.0 pedidos)
   3. COMANDANTE FONTANA → $6,365,000 (2,909.0 pedidos)

RECOMENDACIÓN  (Logística y distribución):
→ Crear rutas diarias/semanales optimizadas para las siguientes 3 localidades:
   1. QUITILIPI
   2. PIRANÉ
   3. COMANDANTE FONTANA
→ Asignar stock dedicado y vehículo exclusivo en estas zonas para reducir tiempos de entrega de 24-48h a mismo día.
→ Implementar alertas automáticas de stock bajo en estas localidades clave.
→ Ofrecer entrega gratis o bonificación a clientes recurrentes en estas zonas para aumentar frecuencia.
